In [1]:
from pywebio import start_server
from pywebio.input import *
from pywebio.output import *
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

matplotlib.rc("font",family='Noto Sans CJK JP')

# 设置全局图表大小
plt.rc('figure', figsize=(16, 12))
plt.rc('axes', titlesize=20)  # 设置轴标题的字体大小
plt.rc('axes', labelsize=16)  # 设置轴标签的字体大小
plt.rc('xtick', labelsize=14)  # 设置x轴刻度标签的字体大小
plt.rc('ytick', labelsize=14)  # 设置y轴刻度标签的字体大小
plt.rc('legend', fontsize=14)  # 设置图例的字体大小

# 一次推荐的歌曲数量
REC_NUM = 10

# 分类数据
df = pd.read_csv('./data/final_classify_data_2.csv')
df['mood'] = df['mood'].astype('category')
df_5 = pd.read_csv('./data/final_classify_data_5.csv')
df_5['mood'] = df_5['mood'].astype('category')

# 聚类数据
df_cluster = pd.read_csv('./data/cluster_data.csv')
df_cluster = df_cluster.drop_duplicates(subset='track_id')

# 时区
time_zone_count = df['time_zone'].value_counts()
time_zone_count_filter = time_zone_count[time_zone_count > 500]
time_zone_list = list(time_zone_count.index)

# 标签
hashtags = df['hashtag'].value_counts()
hashtags = hashtags[hashtags > 100][:30]
hashtags_list = list(hashtags.index)

wuyin_table = pd.read_excel('./data/五声音阶的特征.xlsx', sheet_name='en')
wuyin_table

styled_wuyin_table = """
<style>
    table {
        width: 100%;
        border-collapse: collapse;
    }
    th, td {
        border: 1px solid #ddd;
        padding: 8px;
        text-align: center;
        font-size: 14px;
    }
    td:first-child, th:first-child { /* 应用样式到第一列的所有单元格和标题 */
        font-weight: bold; /* 加粗文本 */
    }
    .text-wrap {
        white-space: normal;
    }
</style>
""" + wuyin_table.to_html(index=False, classes='text-wrap')

In [2]:
import base64
import requests

# Spotify 开发者平台注册的应用程序的客户端 ID 和客户端秘钥
client_id = 'a4a6ca001e074c828ccd20cf9643cba5'
client_secret = 'a4162505cf484701b0ac2d3128c80b27'

# 构造认证请求的 URL
auth_url = 'https://accounts.spotify.com/api/token'

# 构造 HTTP 头部，包含客户端 ID 和客户端秘钥
headers = {
    'Authorization': 'Basic ' + base64.b64encode(f'{client_id}:{client_secret}'.encode()).decode(),
}

# 构造请求参数
data = {
    'grant_type': 'client_credentials',
}

# 发送请求，获取访问令牌
response = requests.post(auth_url, headers=headers, data=data)

# 解析响应，提取访问令牌
token = response.json().get('access_token')

print(token)

# token = 'BQBFB6fPY5tChT-VnAYK6csby640soTQAsRVKWlDh4f6U_IKnxwkNpUdsNtct3u4W9v3P1qAGjigQQueD_smphPjxbmTPjF08l_o4trZS8sZjW4WgeM'
# print(token)

BQC2Uj2vx9o9WIGXuF55OuzRmnJpP2OZ78yKyH_R7TT4RkzF4GFpyY0hczu1eCQLjbSP1I_IfBaifE39mmh6ATahJo1d-M7YKr64X-KoLiUenbxgnGY


In [3]:
# 网络配置

local_ip = '100.78.231.77'
proxies = {
    "http": f"http://{local_ip}:7890",
    "https": f"http://{local_ip}:7890"
}

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 361346 entries, 0 to 361345
Data columns (total 18 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   score          361346 non-null  float64 
 1   hashtag        361346 non-null  object  
 2   lang           361346 non-null  object  
 3   tweet_lang     361346 non-null  object  
 4   time_zone      361346 non-null  object  
 5   month          361346 non-null  int64   
 6   hours          361346 non-null  int64   
 7   track_id       361346 non-null  object  
 8   season         361346 non-null  int64   
 9   day_time       361346 non-null  object  
 10  weekday_index  361346 non-null  int64   
 11  weekend        361346 non-null  bool    
 12  labels         361346 non-null  int64   
 13  score_2        361346 non-null  float64 
 14  score_3        361346 non-null  float64 
 15  mood           361346 non-null  category
 16  mood_label     361346 non-null  int64   
 17  happy     

In [5]:
time_range = {
    'Gong': [(6, 9), (11, 15), (17, 19)],
    'Shang':[(15, 17)],
    'Jue':[(19, 23)],
    'Zhi':[(21, 23)],
    'Yu':[(7, 11)]
}
mood_to_track_type = {
    'disgust': 'Shang',
    'sadness': 'Zhi',
    'neutral': 'Jue',
    'happiness': 'Gong',
    'surprise': 'Yu'
}

columns = ['hashtag', 'tweet_lang', 'time_zone', 'lang', 'day_time']

feature_mapping = {}
for column in columns:
    encoder = LabelEncoder()
    encoder.fit_transform(df[column])
    feature_mapping[column] = dict(zip(encoder.classes_, encoder.transform(encoder.classes_)))

# 标签映射
label_mapping = {0: 'Gong', 1: 'Shang', 2: 'Jue', 3: 'Zhi', 4: 'Yu'}
df_5['labels_wuyin'] = df_5['labels'].map(label_mapping)

In [6]:
# 人脸情绪识别模块
import torch.nn as nn
import torch.nn.functional as F
class DCNN(nn.Module):
    def __init__(self, img_depth, num_classes):
        super(DCNN, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=img_depth, out_channels=256, kernel_size=5, padding=2)
        self.batchnorm1 = nn.BatchNorm2d(256)
        
        self.conv2 = nn.Conv2d(in_channels=256, out_channels=128, kernel_size=5, padding=2)
        self.batchnorm2 = nn.BatchNorm2d(128)
        
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.dropout1 = nn.Dropout(0.4)
        
        self.conv3 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.batchnorm3 = nn.BatchNorm2d(128)
        
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, padding=1)
        self.batchnorm4 = nn.BatchNorm2d(128)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.dropout2 = nn.Dropout(0.4)
        
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, padding=1)
        self.batchnorm5 = nn.BatchNorm2d(256)
        
        self.conv6 = nn.Conv2d(in_channels=256, out_channels=256, kernel_size=3, padding=1)
        self.batchnorm6 = nn.BatchNorm2d(256)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.dropout3 = nn.Dropout(0.5)
        
        # Flattening the output for the dense layer
        self.flatten = nn.Flatten()
        # 正确计算展平后的尺寸
        self.dense1 = nn.Linear(256 * 6 * 6, 128)  # Adjusted size after pooling
        self.batchnorm7 = nn.BatchNorm1d(128)
        self.dropout4 = nn.Dropout(0.6)
        
        self.out_layer = nn.Linear(128, num_classes)

    def forward(self, x):
        x = F.elu(self.batchnorm1(self.conv1(x)))
        x = F.elu(self.batchnorm2(self.conv2(x)))
        x = self.dropout1(self.pool1(x))
        
        x = F.elu(self.batchnorm3(self.conv3(x)))
        x = F.elu(self.batchnorm4(self.conv4(x)))
        x = self.dropout2(self.pool2(x))
        
        x = F.elu(self.batchnorm5(self.conv5(x)))
        x = F.elu(self.batchnorm6(self.conv6(x)))
        x = self.dropout3(self.pool3(x))
        
        x = self.flatten(x)
        x = F.elu(self.batchnorm7(self.dense1(x)))
        x = self.dropout4(x)
        x = self.out_layer(x)
        return x  # 返回 logits 用于计算交叉熵损失

import cv2
import torch as t

device = t.device("cuda" if t.cuda.is_available() else "cpu")

# 加载Haar特征的级联分类器，用于面部检测
detection_model_path = './haarcascade_files/haarcascade_frontalface_default.xml'
emotion_model_path = './models/face_emotion_rec_model_5.pth'
face_detection = cv2.CascadeClassifier(detection_model_path) # 人脸检测模型

# 加载情绪识别模型
emotion_classifier = t.load(emotion_model_path, map_location= device)
EMOTIONS = ['disgust',  'happiness', 'sadness', 'surprise',  'neutral']

def emotion_testing(test_img):
    
    # 定义一个变量来控制是否显示当前帧
    predicted_emotion = None
        
    # 照片转化为RGB灰度图
    gray_img= cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)

    # 检测
    faces_detected = face_detection.detectMultiScale(gray_img, 1.32, 5)

    for (x,y,w,h) in faces_detected:
        # 裁剪人脸并调整格式
        cv2.rectangle(test_img,(x,y),(x+w,y+h),(255,0,0),thickness=3)
        roi_gray=gray_img[y:y+w,x:x+h]
        
        # 转化为48 x 48的灰度图并转化为像素点
        roi_gray=cv2.resize(roi_gray,(48,48))
        img_pixels = np.expand_dims(roi_gray, axis=2)
        
        # 提升维度，将值的范围控制到0~1
        img_pixels = np.expand_dims(img_pixels, axis = 0)
        img_pixels = img_pixels.astype(np.float32)
        img_pixels /= 255
        
        img_pixels = t.from_numpy(img_pixels)
        img_pixels = img_pixels.permute(0, 3, 1, 2)  # Reorder from NHWC to NCHW
        print(f'img_pixels:{img_pixels.shape}')
        # 预测情绪
        # 前向传播得到输出
        with torch.no_grad():  # 关闭梯度计算
            output = emotion_classifier(img_pixels.to(device))
            print(f'output:{F.softmax(output, dim=1)}')
            _, prediction = torch.max(F.softmax(output, dim=1), 1)
        
        predicted_emotion = EMOTIONS[prediction]
        
        cv2.putText(test_img, predicted_emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 3, (0,255,0), 3)
        resized_img = cv2.resize(test_img, (0, 0), fx=0.25, fy=0.25, interpolation=cv2.INTER_NEAREST)
        
    return predicted_emotion, resized_img

In [7]:
# 分类模型模块
import torch
import torch as t
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_heads, num_classes, dim_feedforward=512, num_layers=5):
        super(TransformerModel, self).__init__()
        self.input_dim = input_dim
        self.embedding = nn.Linear(input_dim, dim_feedforward)
        
        # 添加 Batch Normalization 层
        self.batch_norm = nn.BatchNorm1d(dim_feedforward)
        
        # 初始化 Transformer 编码器层
        encoder_layer = nn.TransformerEncoderLayer(d_model=dim_feedforward, nhead=num_heads)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        # Dropout 层
        self.dropout = nn.Dropout(0.3)
        
        # 输出层
        self.output_layer = nn.Linear(dim_feedforward, num_classes)
        
        # ReLU 激活函数
        self.relu = nn.ReLU()

    def forward(self, src):
        # src shape: (batch_size, input_dim)
        src = self.embedding(src)  # (batch_size, dim_feedforward)
        
        # 应用批归一化
        src = self.batch_norm(src)
        
        # 应用 ReLU 激活函数
        src = self.relu(src)
        
        src = src.unsqueeze(1)  # (batch_size, 1, dim_feedforward) - Transformer expects seq_len, batch, dim
        output = self.transformer_encoder(src)
        output = output.squeeze(1)  # (batch_size, dim_feedforward)
        
        # 应用 Dropout
        output = self.dropout(output)
        
        output = self.output_layer(output)
        
        return output

transformer2 = t.load('./models/transformer_not_balance_2.pth')
transformer5 = t.load('./models/transformer_not_balance_5.pth')
print(transformer2)

TransformerModel(
  (embedding): Linear(in_features=15, out_features=512, bias=True)
  (batch_norm): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (transformer_encoder): TransformerEncoder(
    (layers): ModuleList(
      (0-4): 5 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=512, out_features=512, bias=True)
        )
        (linear1): Linear(in_features=512, out_features=2048, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=2048, out_features=512, bias=True)
        (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (dropout): Dropout(p=0.3, inplace=False)
  (output_layer): Linear(in_features=512, out_features=5

In [8]:
# 特征工程相关
def get_time():
        # 获取当前时间戳
    timestamp = time.time()

    # 将时间戳转换为datetime对象
    current_datetime = datetime.fromtimestamp(timestamp)

    # 提取年、月、日、小时和星期几
    year = current_datetime.year
    month = current_datetime.month
    day = current_datetime.day
    hour = current_datetime.hour
    weekday = current_datetime.weekday()  # 注意：这里星期一是0，星期天是6

    return {'year':year, 'month': month, 'day':day, 'hour': hour, 'weekday':weekday + 1}

def classify_time_period(hour):
    if hour < 6:
        return 'Wee hours(00:00 - 6:00)'
    elif hour < 12:
        return 'Morning(6:00 - 12:00)'
    elif hour < 18:
        return 'Afternoon(12:00 - 18:00)'
    else:
        return 'Night(18:00 - 24:00)'

In [ ]:
from pywebio import config
from pywebio import pin
from pywebio.session import *
from functools import partial
import asyncio
import nest_asyncio
import io
import re
import seaborn as sns
import requests
import json
from datetime import datetime
import time
from PIL import Image
import numpy as np

self_sample = None
        
def show_info(sample, index = False):
    # 通过PyWebIO输出HTML
    put_html(sample.to_html(index=index))
    
def show_plt():
    # 图片存入内存
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    buf.seek(0)
    
    
    put_image(buf.read())
    plt.close()

def get_mood(score):
    if score < 0.2:
        return 'disgust'
    elif score < 0.4:
        return 'sadness'
    elif score < 0.6:
        return 'neutral'
    elif score < 0.8:
        return 'happiness'
    else:
        return 'surprise'
    
def set_sample():
    score = pin.pin['score']
    time_zone = pin.pin['time_zone']
    hashtag = pin.pin['hashtag']
    lang = pin.pin['lang']
    t = get_time()
    sample = pd.DataFrame({
        'score' :score,
        'mood': get_mood(score),
        'hashtag':hashtag,
        'lang':lang,
        'tweet_lang':lang,
        'time_zone':time_zone,
        'month':t['month'],
        'hours':t['hour'],
        'season':t['month'] // 4,
        'day_time':classify_time_period(t['hour']),
        'weekday_index':t['weekday'],
        'weekend':(t['weekday'] > 5)
    }, index=[0])
    
    with use_scope('sample', clear = True):
        put_markdown('#### Your sample info：')
        show_info(sample)
        put_markdown('---')
        
    global self_sample
    self_sample = sample

def play_music(music_url):
    audio_html = f"""
    <audio controls>
      <source src="{music_url}" type="audio/mpeg">
      Your browser does not support the audio element.
    </audio>
    """
    put_html(audio_html)
    
def get_time_zone_info():
    if self_sample is None:
        popup('Error', put_error('Please confirm your personal information first!'))
        return
    # 时区人数图
    time_zone_count_filter.plot(kind='bar')
    time_zone = pin.pin['time_zone']
    
    plt.xlabel('Time Zone')
    plt.ylabel('Frequency')
    plt.title('Frequency of Time Zones')
    
    with use_scope('display_zone', clear = True):
        show_plt()
        put_text(f'{time_zone} annual audience is：{time_zone_count[time_zone]}。')
    
def get_time_zone_year():
    if self_sample is None:
        popup('Error', 'Please confirm your personal information first!')
        return
    # 1月份-12月份听歌人数的变化
    data_group_by_month = df.groupby(by=['month', 'time_zone']).size().unstack(fill_value=0).stack().reset_index(name='count')
    data_group_by_month = data_group_by_month[data_group_by_month['time_zone'].isin(time_zone_count_filter.index)]

    # 使用seaborn绘制折线图
    sns.lineplot(data=data_group_by_month, x='month', y='count', hue='time_zone', palette='bright')
    plt.title('Listeners by Time Zone Over Month')
    plt.ylabel('Number of Listeners')
    plt.xlabel('Month')
    plt.legend(loc='best')
    plt.xticks(rotation=45)
    
    with use_scope('display_zone', clear = True):
        show_plt()
        
def get_hashtag_info():
    if self_sample is None:
        popup('Error', 'Please confirm your personal information first!')
        return
    fig, axes = plt.subplots(1, 1)
    hashtags.plot.barh(ax=axes, color='#2ecc71', alpha=0.7) # 翠绿
    plt.xlabel('Frequency')
    axes.set_title('The frequency of hashtags')
    hashtag = pin.pin['hashtag']
    with use_scope('display_zone', clear = True):
        show_plt()
        put_text(f'There are {hashtags[hashtag]} users tagging #{hashtag}：。')
        
def set_initial_info():
    pin.put_slider('score', label='Input your sentiment score(0 - 1，negetive - positive)', min_value=0.0, max_value=1.0, step=0.1, value=0.8, required=True),
    pin.put_select('time_zone', label = 'time zone', options=time_zone_list, value = 'Beijing'),
    pin.put_select('hashtag', label = 'your hashtag', options=hashtags_list, value = 'np'),
    pin.put_radio('lang', label='your language', options=['en', 'nl', 'zh-tw', 'zh-Hans'], value = 'zh-Hans')
    put_buttons(['submit', 'Display time distribution', 'Show listeners', 'Displays listening trends', 'Display label'], 
                onclick=[set_sample, get_day_time_info, get_time_zone_info, get_time_zone_year, get_hashtag_info])
    
def get_day_time_info():
    if self_sample is None:
        popup('Error', 'Please confirm your personal information first!')
        return
    
    period = df[df['time_zone'] == self_sample['time_zone'][0]]['day_time']
    period = period.value_counts()
    fig, axes = plt.subplots(1, 1)
    axes.pie(period.values, labels=period.index, autopct='%1.1f%%', shadow=True, startangle=140, colors=['#3498db', '#9b59b6', '#e74c3c','gray' ])
    axes.set_title(f'Time Distribution of {self_sample["time_zone"][0]} in one day')
    axes.set_ylabel('')
    with use_scope('display_zone', clear = True):
        show_plt()
        put_text(f'In your time zone { self_sample["time_zone"][0] }, the listening time distribution is as above')
        
def get_tracks(track_ids):
    # Spotify API 请求的 URL
    url = 'https://api.spotify.com/v1/tracks'
    
    # 将 track ID 列表转换为逗号分隔的字符串，作为请求参数
    params = {
        'ids': ','.join(track_ids)
    }
    
    # 构造请求头，包含访问令牌
    headers = {
        'Authorization': f'Bearer {token}',
    }

    # 发送 GET 请求，获取 track 的信息
    response = requests.get(url, headers=headers, params=params)
    
    # 解析响应，获取 track 的信息
    tracks = response.json()
    print(tracks)
    tracks = tracks['tracks']
    
    return tracks

def get_playlist_html(tracks):
    html_code = """
    <!DOCTYPE html>
    <html lang="en">
    <head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>My Music Playlist</title>
    <style>
    body {
        font-family: Arial, sans-serif;
        background-color: #f0f0f0;
        margin: 0;
        padding: 0;
    }

    .container {
        max-width: 800px;
        margin: 20px auto;
        padding: 20px;
        background-color: #fff;
        box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
    }

    .song {
        display: flex;
        align-items: center;
        justify-content: space-between;
        border-bottom: 1px solid #ddd;
        padding: 10px 0;
    }

    .song img {
        width: 120px;
        height: 120px;
        margin-right: 20px;
    }

    .song-details {
        flex-grow: 1;
    }

    .song-title {
        font-size: 1.2em;
        font-weight: bold;
        margin: 0 0 5px;
    }

    .song-artist {
        font-size: 1em;
        margin: 0 0 5px;
    }

    .song-album {
        font-size: 0.9em;
        color: #666;
        margin: 0 0 5px;
    }

    .song-release-date {
        font-size: 0.9em;
        color: #666;
        margin: 0 0 5px;
    }

    .play-button {
        padding: 5px 10px;
        background-color: #1db954;
        color: #fff;
        border: none;
        border-radius: 5px;
        cursor: pointer;
    }

    .play-button:hover {
        background-color: #1ed760;
    }
    </style>
    </head>
    <body>
    <div class="container">
    """
    
    for track in tracks:
        song_html = f"""
        <div class="song">
            <img src="{track['album']['images'][0]['url']}" alt="Album Image">
            <div class="song-details">
                <div class="song-title">{track['name']}</div>
                <div class="song-artist">artist: {track['artists'][0]['name']}</div>
                <div class="song-album">album: {track['album']['name']}</div>
                <div class="song-release-date">release date: {track['album']['release_date']}</div>
                <div class="song-url">{track['external_urls']['spotify']}</div>
            </div>
        </div>
        """
        html_code += song_html

    html_code += """
    </div>
    </body>
    </html>
    """
    
    return html_code

def get_preview_url(url):
    try:
        # 向指定的 URL 发送 GET 请求，通过代理进行
        response = requests.get(url ,proxies=proxies)
        
        # 定义正则表达式模式，这里是找了音乐片段的url，若要全音频，需要另想办法
        pattern = r'<meta property="og:audio" content="(https://[^"]+)"'
        
        # 使用正则表达式查找匹配的内容
        match = re.search(pattern, response.text)
        if match:
            audio_url = match.group(1)
            return audio_url
            
        if response.status_code == 200:
            print("Get_preview_url successful!")
        else:
            print("Failed to reach Spotify.")
    except requests.exceptions.RequestException as e:
        print("Error during requests to {0} : {1}".format(url, str(e)))
    
def get_recs(df, based, shuffle = False, rec_label = None, predicted = None):

    group_df = df.groupby(by=['track_id']).count()
    # 选择第一列
    first_column = group_df.iloc[:, 0]  # 保留第一列

    # 创建一个新的 DataFrame 并重命名列为 'count'
    rec_df = pd.DataFrame(first_column).rename(columns={first_column.name: 'count'})
    
    # 按 'count' 列从大到小排序
    rec_df = rec_df.sort_values(by='count', ascending=False)

    if shuffle:
        rec_df = rec_df.sample(n=REC_NUM)
    # 指定要获取信息的 track ID 列表
    track_ids = rec_df.index[:REC_NUM]
    
    tracks = get_tracks(track_ids)
    
    # 打印 track 的信息
    print(tracks)
    
    html_code = get_playlist_html(tracks)
    
    with use_scope('playlist', clear = True):
        if based == 'zone':
            time_zone = self_sample['time_zone'][0]
            put_markdown(f'**Your time zone is { time_zone }，recommend {REC_NUM} tracks for you based on your mood.**')
        elif based == 'time':
            day_time = self_sample['day_time'][0]
            hour = self_sample['hours'][0]
            put_markdown(f"**The time is { day_time.split('(')[0] }, {hour} o'clock，recommend {REC_NUM} tracks for you based on your mood.**")
        elif based == 'hashtag':
            tag = self_sample['hashtag'][0]
            put_markdown(f'**You choose the tag #{tag}, recommend {REC_NUM} tracks for you based on your mood.**')
        elif based == 'emotion_2':
            mood = self_sample['mood'][0]
            put_markdown(f'**Detect your emotions as { mood }, recommend {REC_NUM} tracks for you based on your mood.**')
        elif based == 'emotion_5':
            mood = self_sample['mood'][0]
            put_markdown(f'**Detect your emotions as { mood }, model recommends {REC_NUM} tracks mode {label_mapping[predicted]} for you.**')
        elif based == 'season':
            season = self_sample['season'][0]
            month = self_sample['month'][0]
            season_map = {0: 'Spring', 1: 'Summer', 2: 'Autumn', 3: 'Winter'}
            put_markdown(f'**The month is { month }, season is { season_map[season] }, recommend {REC_NUM} tracks for you based on your mood.**')
        elif based == 'wuyin_daytime':
            hour = self_sample['hours'][0]
            put_markdown(f"**The time is：{hour} o'clock.Suitable for listening to：{rec_label}**")
        elif based == 'regulation_based':
            mood = self_sample['mood'][0]
            labels_wuyin = mood_to_track_type[mood]
            put_markdown(f'**Detect your emotions as { mood }.Suitable for listening to {labels_wuyin}, recommend {REC_NUM} tracks for you based on your mood.**')
            
        put_html(html_code)
        put_markdown('---')
        
    # 滚动到列表开头    
    scroll_to('playlist')
    
    playlist = [{
        'image_url': track['album']['images'][0]['url'], 
        'title' : track['name'],
        'artist': track['artists'][0]['name'],
        'audio_url': get_preview_url(track['external_urls']['spotify']),
        'lyric': 'To respect the copyright of the song, only 30s music clips are played. \n For full music, please go to the link in the song list'
    } for track in tracks]
    
    with use_scope('playlist'):
        put_buttons(['Play'], onclick=[partial(music_player, playlist=playlist)])
        
    scroll_to(position = 'bottom')
        
def zone_based_rec():
    # 处理数据，获得时区前10
    time_zone = self_sample['time_zone'][0]
    mood = self_sample['mood'][0]
    df_zone = df[df['time_zone'] == time_zone]
    
    print(df_zone)
    # 这里主要是防止选择平静的用户，平静的用户相对较少，这里可能出现空的情况
    if (df_zone['mood'] == mood).sum() >= 10:
        df_zone = df_zone[df_zone['mood'] == mood]
    
    print(df_zone)
    get_recs(df_zone, 'zone')
    
def time_based_rec():
    hour = self_sample['hours'][0]
    day_time = self_sample['day_time'][0]
    mood = self_sample['mood'][0]

    df_time = df[df['day_time'] == day_time]
    
    # 这里主要是防止选择平静的用户，平静的用户相对较少，这里可能出现空的情况
    if (df_time['mood'] == mood).sum() >= 10:
        df_time = df_time[df_time['mood'] == mood]

    get_recs(df_time, 'time')
    
def tag_based_rec():
    tag = self_sample['hashtag'][0]
    mood = self_sample['mood'][0]
    df_tag = df[df['hashtag'] == tag]
    
    # 这里主要是防止选择平静的用户，平静的用户相对较少，这里可能出现空的情况
    if (df_tag['mood'] == mood).sum() >= 10:
        df_tag = df_tag[df_tag['mood'] == mood]
        
    get_recs(df_tag, 'hashtag')
def season_based_rec():
    season = self_sample['season'][0]
    mood = self_sample['mood'][0]
    df_season = df[df['season'] == season]
    
    # 这里主要是防止选择平静的用户，平静的用户相对较少，这里可能出现空的情况
    if (df_season['mood'] == mood).sum() >= 10:
        df_season = df_season[df_season['mood'] == mood]
    
    get_recs(df_season, 'season')
    
def total_rec():
    pass

def get_player_html(playlist):
    html_content = f"""
    <html>
    <head>
        <style>
            .player-container {{
                background-color: #f0f0f0;
                padding: 20px;
                width: 100%;
                max-width: 600px;
                margin: auto;
                box-shadow: 0 4px 8px rgba(0,0,0,0.1);
                text-align: center;
                font-family: Arial, sans-serif;
            }}
            button {{
                margin: 5px;
                padding: 5px 20px;
                font-size: 16px;
                background-color: #4CAF50;
                color: white;
                border: none;
                border-radius: 5px;
                cursor: pointer;
            }}
            button:hover {{
                background-color: #45a049;
            }}
            #trackInfo {{
                margin-top: 10px;
                font-size: 18px;
                color: #333;
            }}
            .lyrics {{
                text-align: center;  /* 文字居中显示 */
                font-size: 20;     /* 文字大小 */
                line-height: 36px;   /* 行高 */
                margin-top: 10px;   /* 顶部间距 */
                white-space: pre-line; /* 换行 */
            }}
        </style>
    </head>
    <body>
        <div class="player-container">
            <img id="trackImage" src="{playlist[0]['image_url']}" style="width: 200px; height: 200px; border-radius: 10px; margin-bottom: 10px;">
            <div id="trackInfo">{playlist[0]['title']}</div>
            <audio id="audioPlayer" controls autoplay>
                <source src="{playlist[0]['audio_url']}" type="audio/mpeg">
                Your browser does not support the audio tag.
            </audio>
            <br>
            <button onclick="previousTrack()">Previous</button>
            <button onclick="nextTrack()">Next</button>
            <div class="lyrics", id="lyric">
                {playlist[0]['lyric']}
            </div>
            
        </div>
        <script>
            var playlist = {playlist};
            var currentTrack = 0;
            var player = document.getElementById('audioPlayer');
            var trackInfo = document.getElementById('trackInfo');
            var trackImage = document.getElementById('trackImage')
            var lyric = document.getElmentById('lyric')

            function setTrack(track) {{
                player.src = track['audio_url'];
                player.load();
                player.play();
                trackInfo.innerText = track['title'];
                trackImage.src = track['image_url'];
                lyric.innerText = track['lyric'];
            }}

            function nextTrack() {{
                if (currentTrack < playlist.length - 1) {{
                    currentTrack++;
                }} else {{
                    currentTrack = 0; // Loop back to the first track
                }}
                setTrack(playlist[currentTrack]);
            }}

            function previousTrack() {{
                if (currentTrack > 0) {{
                    currentTrack--;
                }} else {{
                    currentTrack = playlist.length - 1; // Loop to the last track
                }}
                setTrack(playlist[currentTrack]);
            }}
        </script>
    </body>
    </html>
    """
    return html_content

def music_player(playlist):
    # 将 Python 列表转换为 JavaScript 数组格式的字符串
    html_content = get_player_html(playlist)
    with use_scope('musicbar', clear = True):
        put_html(html_content)

def play_track():
    url = "https://p.scdn.co/mp3-preview/3ff56360de277b2cc32d713e0798d1ff63b8e8a8?cid=cfe923b2d660439caf2b557b21f31221"
    audio_html = f"""
    <audio controls>
        <source src="{url}" type="audio/mp3">
        Your browser does not support the audio element.
    </audio>
    
    """
    put_html(audio_html)
def clear_all():
    clear('page1')
    clear('page2')
    clear('page3')
    clear('page4')
    clear('display_zone')
    clear('playlist')
    clear('musicbar')
    
def page1():
    clear_all()
    with use_scope('page1', clear=True):
        set_initial_info()

def page2():
    if self_sample is None:
        popup('Error', 'Please confirm your personal information first!')
        return
    clear_all()
    with use_scope('display_zone', clear=True):  # 清除页面内容作用域中的旧内容
        put_buttons(['Time_zone based rec', 'Daytime based rec', 'Tag based rec', 'Season based rec'], 
                onclick=[zone_based_rec, time_based_rec, tag_based_rec, season_based_rec])

def show_cluster_img(k = 2):
    X = StandardScaler().fit_transform(df_cluster.drop(['track_id'], axis = 1))

    kmeans = KMeans(init='k-means++', n_clusters = k)
    kmeans.fit(X)
    y_kmeans = kmeans.predict(X)

    # 可视化3D聚类结果
    fig = plt.figure(figsize=(16, 12))
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(X[:, 2], X[:, 5], X[:, 7], c=y_kmeans, cmap='viridis')

    # 绘制聚类中心
    centers = kmeans.cluster_centers_
    ax.scatter(centers[:, 2], centers[:, 5], centers[:, 7], c='black', s=200, alpha=0.5)

    fontsize = 16
    ax.set_title(f'Result of the {k} classification', fontsize = fontsize + 4)
    ax.set_xlabel('valence', fontsize=fontsize)
    ax.set_ylabel('energy', fontsize=fontsize)
    ax.set_zlabel('key', fontsize=fontsize)
    
    show_plt()
    put_markdown('---')

def get_emotion():
    # todo 通过摄像头或者图片分析情绪
    # 文件上传
    with use_scope('playlist', clear = True):
        file = file_upload("Upload a photo to recognize your emotion", accept="image/*")
    if file is not None:
        # 预处理图片
        image = Image.open(io.BytesIO(file['content']))
        image = np.array(image)
        
        emotion_word, resized_img = emotion_testing(image)
        print(f'Emotion detected is {emotion_word}')
        
        image_pil = Image.fromarray(resized_img)
        with use_scope('playlist', clear = True):
            put_image(image_pil)
            put_markdown('**Wait for a moment......**')
        
        time.sleep(3)
        return emotion_word
    
    return 'happiness'

def feature_project(X):
    ''' 这里做和训练集一样的特征工程 '''
    from sklearn.preprocessing import StandardScaler, MinMaxScaler
    # 对分数score取平方、3次方
    X['score_2'] = X['score'] ** 2
    X['score_3'] = X['score'] ** 3

    # 这里从训练集获得mood映射标签的值
    mood_categories = df['mood'].cat.categories
    
    # 使用与训练集相同的分类映射
    X['mood'] = X['mood'].astype('category')
    X['mood'] = pd.Categorical(X['mood'], categories=mood_categories)
    X['mood_label'] = X['mood'].cat.codes
    X.drop('mood', inplace = True, axis = 1)
    
    bins = [0.0, 0.5, 1.0]
    X['happy'] = pd.cut(X['score'], labels=[0, 1], bins = bins, include_lowest=True)
    
    print(X)
    
   # 这里需要按训练集编码映射一下样本的特征编码
    for column in columns:
        X[column] = X[column].map(feature_mapping[column])
    
    X = MinMaxScaler().fit_transform(X)
    X = StandardScaler().fit_transform(X)
    X_tensor = t.tensor(X, dtype=t.float32)
    return X_tensor.to(device)

def get_emotion_rec(k):
    # todo： 载入我的情绪识别模型，识别情绪
    mood = get_emotion()
    
    self_sample['mood'] = mood
    
    X_test = feature_project(self_sample.copy())
    
    model = transformer2
    based = 'emotion_2'
    data = df
    if k == 5:
        model = transformer5
        based = 'emotion_5'
        data = df_5
        
    model.eval()
    predicted = 0
    with t.no_grad():
        
        y_pred = model(X_test)
        # 处理预测结果
        predicted = torch.argmax(y_pred, dim=1)  # 获取最大 logit 的索引，即预测的类别
    
    print(f'predicted:{predicted.item()}')
    df_emotion = data[data['labels'] == predicted.item()]
        
    get_recs(df_emotion, based = based, shuffle = True, predicted = predicted.item())

def show_two_page():
    clear_all()
    close_popup()

    with use_scope('playlist', clear = True):
        show_cluster_img(k = 2)
        put_buttons(['Emotion Input'], onclick=[partial(get_emotion_rec, k = 2)])

def day_time_based_rec():
    #这里是按宫商角徵羽来推的
    hour = self_sample['hours'][0]
    
    rec_label = set()
    for label, times in time_range.items():
        for start, end in times:
            if hour >= start and hour < end:
                rec_label.add(label)

    rec_df = df_5[df_5['labels_wuyin'].isin(rec_label)]
    get_recs(rec_df, 'wuyin_daytime', shuffle = True, rec_label = rec_label)
    
def regulation_based_rec():
    mood = self_sample['mood'][0]
    labels_wuyin = mood_to_track_type[mood]
    
    df_mood = df_5[df_5['labels_wuyin'] == labels_wuyin]

    get_recs(df_mood, 'regulation_based', shuffle = True)

def show_five_page():
    clear_all()
    close_popup()
    
    # 使用 replace() 方法进行映射
    df['labels'] = df['labels'].replace(label_mapping)
    
    with use_scope('display_zone', clear = True):
        put_html(styled_wuyin_table)
        put_text('Table collated from')
        put_link('1、NetEase articles', 'https://ds.163.com/article/5faa98d2f7feba4623704eae/', new_window = True)
        put_text('')
        put_link('2、Thesis', 'https://kns.cnki.net/kcms2/article/abstract?v=29axctaKF3zEV--IRp7m2uQIKb9z2zY76Yu3JRRy7OprmsI-LS0sP_h0ykdBtF_CYbvqe0CkuZEg6yD04P3-1Rfliy9vFo-ygRt47aunFCYDxBirmwWeIv22prGVAX0K7Z9AqXYdptY=&uniplatform=NZKPT&flag=copy', new_window = True)
        put_markdown('---')
        put_buttons(['Daytime based rec', 'Regulation based rec', 'Emotion Input'], 
            onclick=[day_time_based_rec, regulation_based_rec,  partial(get_emotion_rec, k = 5)])
    with use_scope('playlist', clear = True):
        show_cluster_img(k = 5)
        
def page3():
    if self_sample is None:
        popup('Error', 'Please confirm your personal information first!')
        return
    
    clear_all()
    # 创建一个弹窗询问用户
    popup("Choose the classification num", [
        put_buttons(['two', 'five'], onclick=[show_two_page, show_five_page])
    ])
    
def page4():
    if self_sample is None:
        popup('Error', 'Please confirm your personal information first!')
        return
    
    clear_all()
    clear('sample')
    
    BASE_URL = 'http://10.80.43.30:3000/api/'
    GEN_API = 'generate'
    
    info = None
    with use_scope('page4', clear = True):
        info = input_group("Music Gen",[
          textarea('Please describe your mood within 500 words, the more description, the richer the content!', rows = 10, name='prompt'),
          radio('Pure music or with lyric', options=[('Pure music', True), ('With lyric', False)], required=True, name='instrumental')
        ])
    
    prompt = info['prompt']
    instrumental = info['instrumental']
    
    print(prompt, '\n', instrumental)
        
    # 调用suno AI 的 API 生成音乐
    request_body = {
      "prompt": prompt,
      "tags": self_sample['hashtag'][0],
      "make_instrumental": instrumental,
      "wait_audio": True
    }
    
     # 显示加载标识，来自（output模块）
    with put_loading():
        put_text("Generating music. Please waiting for a minute......")
        response = requests.post(BASE_URL + GEN_API, json = request_body)

    # 打印响应体（内容）
    track_list = json.loads(response.text)
    print(track_list)

    with use_scope('playlist', clear=True): 
        put_success('Successful generating. Recommendate two tracks for you. Enjoying it!', closable=True)
        music_player(playlist=track_list)
        
def print_sample():
    print(self_sample)
def navigation_bar():
    with use_scope('navbar', clear=True):  # 使用独立的作用域，并在更新时清除旧内容
        put_buttons(['Basic Info', 'Manual Rec', 'Model Rec', 'Music Gen', 'print_sample'], onclick=[page1, page2, page3, page4,print_sample])
        put_markdown('---')
        
@config(theme="minty")
def main():
    """EBMusic"""
    navigation_bar()  # 初始时设置导航条
    page1()  # 默认显示第一页内容
        

if __name__ == '__main__':
    nest_asyncio.apply()
    asyncio.create_task(start_server(main, port=9999, debug=True, notebook=True))
    

Running on all addresses.
Use http://10.80.43.30:9999/ to access the application
      score     mood hashtag lang tweet_lang time_zone  month  hours  season  \
0  0.224364  sadness      np   en         en    Berlin      5     10       1   

                day_time  weekday_index  weekend  
0  Morning(6:00 - 12:00)              4    False  
   score      mood hashtag     lang tweet_lang time_zone  month  hours  \
0    0.8  surprise      np  zh-Hans    zh-Hans   Beijing      5     10   

   season               day_time  weekday_index  weekend  
0       1  Morning(6:00 - 12:00)              4    False  


宫调：浑和厚重如土，旋律悠扬
颜色：#8B4513 (土褐色)
理由：土褐色象征大地和稳重，与宫调的厚重和悠扬特性相匹配。
商调：初秋清凉、悲伤
颜色：#4682B4 (钢青色)
理由：钢青色给人一种清新凉爽的感觉，适合表达商调的清凉和悲伤情感。
角调：清脆如木，曲调亲切
颜色：#32CD32 (鲜绿色)
理由：鲜绿色代表新生和活力，象征木的生命力和亲切感，与角调的特点吻合。
徵调：热烈如火，节奏欢快
颜色：#FF4500 (橙红色)
理由：橙红色热情且引人注目，能很好地代表徵调的热烈和欢快。
羽调：苍凉淡荡如水，风格清纯
颜色：#ADD8E6 (淡蓝色)
理由：淡蓝色给人一种宁静和清纯的感觉，适合羽调的苍凉和清纯特性。